# Ejercicio : Clústering con scikit-learn/pyspark.ml.clustering

`scikit-learn` (o `sklearn`) es una librería que reúne muchas herramientas para realizar Minería de Datos y _Aprendizaje de Máquinas_. Permite hacer clasificación, clustering, entre otras. Además, incluye varios datasets para aprender a usar la librería.

En este Ejercicio se refuerza los conceptos de _aprendizaje no supervisado_ y a mostrar cómo usar `sklearn` y `ml.clustering` para entrenar nuestro primer Clústering.

Puedes ejecutar cada una de las celdas de código haciendo click en ellas y presionando `Shift + Enter`. 

También puedes editar cualquiera de estas celdas. Las celdas no son independientes. Es decir, sí importa el orden en el que las ejecutes, y cualquier cambio que hagas se reflejará en las celdas que ejecutes después.

---

Cargamos el digit Dataset que viene en `sklearn`.Cada punto de datos es una imagen de 8x8 de un dígito.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [ ]:
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd

X_digits, y_digits = load_digits(return_X_y=True)
#data = scale(X_digits)
n_samples, n_features = X_digits.shape
n_digits = len(np.unique(y_digits)) # Retorna los valores únicos dentro del vector, en este caso (y_digits)
label = y_digits
print(X_digits.shape)
print(y_digits.shape)

#sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))

x = pd.DataFrame(X_digits)

Teniendo los datos cargados vamos a crear la instancia de spark, utilizando `findspark`, donde podemos comprobar la ruta de donde se carga el spark con `findspark.find()`.

In [ ]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="digits")

from pyspark.sql import SQLContext 
sq = SQLContext(sc)
findspark.find()

En este caso es necesario cargar en RDD los datos, lo hacemos para mostrar como podemos cambiar un tipo `pandas` a un tipo `rdd`

In [ ]:
rdd = sc.parallelize(X_digits)
rdd2 = sc.parallelize(y_digits)
print(rdd.take(2))
print(rdd2.take(2))


Creamos un dataframe en base a los rdd cargados, utilizamos la componente `rdd.map`

In [ ]:
data = rdd.map(lambda r: r.tolist()).toDF()

print(data.count())
data.printSchema()

# Se comienza el exploración y transformación de datos

Preparamos la data para lograr realizar clustering, esto porque la componente `KMeans` solicita una entrada de `features` que corresponde a un vector con todas las características que se quiere entrenar.

Entonces,
    para lograr el vector de `features` utilizamos la componente `VectorAssembler` para elos datos.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

vector_features = VectorAssembler(inputCols=[x for x in data.columns], outputCol="features")
Total_Clus_NS = vector_features.transform(data)
print(Total_Clus_NS)
     

Ahora se revisa si en el set de datos vienen datos nulos para las variables, dado que el algoritmo necesita no contar con valores nulo, entonces en caso que haya nulos se debe:

1.- Eliminar variables del estudio si la cantidad de nulos es muy alta que no se pueda determinar un valor de reemplazo

2.- Tratar los valores nulos de las variables, reemplazando estos valores por el valor mas ad-hoc considerando los demás datos de la variable


In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
missing_df = data.select([c for c in data]).toPandas().isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,4))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Cantidad de valores nulos")
ax.set_title("Número de valores nulos por cada variable")
plt.show()

Como se puede visualizar no tenemos valores nulos, por lo que podemos continuar con el estudio.

El siguiente paso es realizar la normalización de las variables.

# ¿Qué es la nomalización?

`La normalización` es una técnica que se aplica a menudo como parte de la preparación de datos para el aprendizaje automático. El objetivo de la normalización es cambiar los valores de las columnas numéricas del conjunto de datos para usar una escala común, sin distorsionar las diferencias en los intervalos de valores ni perder información. La normalización también es necesaria para que algunos algoritmos modelen los datos correctamente.

Por ejemplo, suponga que el conjunto de datos de entrada contiene una columna con valores comprendidos entre 0 y 1, y otra columna con valores comprendidos entre 10 000 y 100 000. La enorme diferencia en el escala de los números podría generar problemas al intentar combinar los valores como características durante el modelado.

La normalización evita estos problemas mediante la creación de nuevos valores que mantienen la distribución general y las relaciones en los datos de origen, a la vez que se conservan los valores dentro de una escala que se aplica en todas las columnas numéricas que se usan en el modelo.

# Nuestra normalización:

El método que se utilizará para normalizar es el `StandardScaler` de la componente `pyspark.ml.feature`. Este método estadisticamente realiza la normalización de la siguiente manera:

`StandardScaler` transforma un conjunto de datos de filas de vectores, normalizando cada característica para que tenga una desviación estándar de la unidad y/o media cero. Toma parámetros:

1.- withStd: True por defecto. Escala los datos a la desviación estándar de la unidad.

2.- withMean: False por defecto. Centra los datos con media antes de escalar. Generará una salida densa, así que tenga cuidado cuando aplique a entradas dispersas.

`StandardScaler` es un Estimador que puede ajustarse en un conjunto de datos para producir un StandardScalerModel; esto equivale a calcular estadísticas resumidas. El modelo puede transformar una columna de Vector en un conjunto de datos para tener desviaciones estándar de la unidad y/o características de media cero.

Nota: Tener en cuenta que si la desviación estándar de una característica es cero, devolverá el valor predeterminado de 0.0 en el Vector para esa característica.

Mas formas de normalizar se pueden revisar en: https://spark.apache.org/docs/latest/ml-features#standardscaler

In [ ]:
from pyspark.ml.feature import StandardScaler

standardizer = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='std_features')
model = standardizer.fit(Total_Clus_NS)
Total_Clus = model.transform(Total_Clus_NS)

# Análisis de la cantidad de clúster óptima

Comenzamos el análisis de cual es el mejor cantidad de clúster, para ello no ayuda el análisis de:

1.- `elbow` (codo), que nos indica cuando jústamente se grafica un códo la cantidad óptima de clúster.

2.- `Silhouette` mide la calidad del agrupamiento o clustering con la distancia de separación entre los clústers. Nos indica como de cerca está cada punto de un clúster a puntos de los clústers vecinos. Esta medida de distancia se encuentra en el rango `[-1, 1]`. Un valor alto indica un buen clustering.

Se comienza con el análisis de `elbow`

In [ ]:
from pyspark.ml.clustering import KMeans
import numpy as np
from pyspark.ml.evaluation import ClusteringEvaluator

cost = np.zeros(17)
silhouette = np.zeros(17, dtype=float)
for k in range(2,17):
    kmeans = KMeans()\
            .setK(k)\
            .setSeed(1) \
            .setFeaturesCol("std_features")

    model = kmeans.fit(Total_Clus)
    cost[k] = model.computeCost(Total_Clus) 
    predictions_clus = model.transform(Total_Clus)
    evaluator = ClusteringEvaluator()
    silhouette[k] = evaluator.evaluate(predictions_clus)

Graficar la curva del `elbow` para validar el número óptimo de clúster

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sbs
from matplotlib.ticker import MaxNLocator

fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,17),cost[2:17], 'b-', marker = 'o', markersize=8, lw=2)
ax.grid(True)
ax.set_xlabel('clúster')
ax.set_ylabel('costo')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.show()

Para asegurarnos del número óptimo hacemos el análisis de `Silhouette`, con su correspondiente gráfica

In [ ]:
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,17), silhouette[2:17], 'b*-')
ax.plot(range(2,17), silhouette[2:17], marker='o', markersize=8, 
         markeredgewidth=2, markeredgecolor='r', lw=2, markerfacecolor='None')
ax.grid(True)
ax.set_xlabel('Número de clúster')
ax.set_ylabel('Coeficiente de Silhouette')
ax.set_title('Silhouette Scores for k-means clustering')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
display(fig)

# Resultado Análisis

Revisando las curvas de `Elbow` y `Silhouette`, vemos que lo mas óptimo podría ser trabajar con `8` clúster, aunque también podría ser interesante trabajar con `14`. Si bien sabemos que el total de números diferentes es `10`, nuestro análisis muestra que no sería la mejor opción, aunque como podemos ver en las curvas el análisis no es totalmente determinante.

Entonces primero vamos ejecutar `8` clúster.

# Reducción de Dimensionalidad

Como vemos el dataset cuenta con 64 columnas, eso indica que estamos ejecutando el clúster con 64 `features`, lo que genera que el algoritmo evalúe 64 entradas distintas para lograr determinar en este caso el clúster a que pertenece.

Una de las técnicas de preprocesado para modelos de aprendizaje es la reducción de la dimensionalidad, que no es más que la reducción del número de variables en una colección de datos.

Las razones por las que nos interesa reducir la dimensionalidad son varias:

1.- Porque interesa identificar y eliminar las variables irrelevantes.

2.- Porque no siempre el mejor modelo es el que más variables tiene en cuenta.

3.- Porque se mejora el rendimiento computacional, lo que se traduce en un ahorro en coste y tiempo.

4.- Porque se reduce la complejidad, lo que lleva a facilitar la comprensión del modelo y sus resultados.


# Reducción de Dimensionalidad con componentes principales

Entre metodologías principales de reducción de dimensionalidad se encuentra la reducción lineal, donde se tien a las `PCA o los componentes principales`, estas son ciertamente las principales herramientas del Statistical Machine Learning.

El análisis de los `componentes principales` es el punto de partida para muchos análisis (y como una herramienta de preprocesamiento) y su conocimiento se vuelve imperativo en caso que las condiciones en la linealidad sean satisfactorios.

En estadística, el análisis de componentes principales es una técnica utilizada para describir un conjunto de datos en términos de nuevas variables (`componentes`) no correlacionadas. Los componentes se ordenan por la cantidad de varianza original que describen, por lo que la técnica es útil para reducir la dimensionalidad de un conjunto de datos.

Técnicamente, el `PCA` busca la proyección según la cual los datos queden mejor representados en términos de mínimos cuadrados. Esta convierte un conjunto de observaciones de variables posiblemente correlacionadas en un conjunto de valores de variables sin correlación lineal llamadas componentes principales.


# ¿Como elegimos la cantidad de componentes principales?

Por lo general, dada una matriz de datos de dimensiones n x p, el número de componentes principales que se pueden calcular es como máximo de n-1 o p (el menor de los dos valores es el limitante). Sin embargo, siendo el objetivo del PCA reducir la dimensionalidad, suelen ser de interés utilizar el número mínimo de componentes que resultan suficientes para explicar los datos. No existe una respuesta o método único que permita identificar cual es el número óptimo de componentes principales a utilizar. Una forma de proceder muy extendida consiste en evaluar la proporción de varianza explicada acumulada y seleccionar el número de componentes mínimo a partir del cual el incremento deja de ser sustancial.


Mas detalles se puede revisar en: 
https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales

y

https://es.wikipedia.org/wiki/Vector_propio_y_valor_propio

Entonces Calulamos y graficamos componentes principales para todos los features del set de datos, consideramos los `64`.

Si revisamos el detalle, el incremento deja de ser sustancial en la componente `24`, pero necesitamos realmente una reducción sustancial de la dimensionalidad dejando la menor cantidad de variables relevantes, entonces volvemos a mirar y podemos decir que tenemos un máximo aproximado de `0,12`, entonces para mi las componentes principales son las que tengan la variable explicada mayor o igual al `50%` del valor máximo, es decir mayor o igual que `0,06`, por ello se es recomendable elegir los `5` componentes principales o menos, en nuestro caso nos quedaremos con los `2` componentes que impactan mas a la varianza acumulada.

In [ ]:
from pyspark.ml.feature import PCA
kp = 64
pca_i = PCA(k=kp, inputCol="std_features", outputCol="pca")
modelPCA_i = pca_i.fit(Total_Clus)
reduced_data_S_i = modelPCA_i.transform(Total_Clus).select('pca')

plt.figure(figsize=(7,7))
eigenvalues = [float(value) for value in modelPCA_i.explainedVariance]
plt.bar(list(range(1,len(eigenvalues)+1)),eigenvalues)
#plt.bar(list(eigenvalues, range(1,len(eigenvalues)+1)))
pd.DataFrame(eigenvalues,columns=['Varianza Explicada'])

# Ejecución de clústering con el análisis realizado y reducción de dimensionalidad

Ejecución con `2` componentes principales y obtenido dicha reducción de dimensionalidad se ejecuta `K-means` con `8 clúster`

In [ ]:
from pyspark.ml.feature import PCA
kp = 2
pca = PCA(k=kp, inputCol="std_features", outputCol="pca")
modelPCA = pca.fit(Total_Clus)
reduced_data_S = modelPCA.transform(Total_Clus).select('pca')

In [ ]:
k = 8
kmeansF = KMeans()\
            .setK(k)\
            .setSeed(1) \
            .setFeaturesCol("pca")

modelKmeans = kmeansF.fit(reduced_data_S)
predictions_reduc = modelKmeans.transform(reduced_data_S)

print(predictions_reduc.show())


# Graficando los clúster encontrados

Para obtener una mejor visualización de los clúster encontrados, se realizan gráficos con la componentes  `mpl_toolkits.mplot3d` y `matplotlib.pyplot`. Para ello se debe:

1.- Transformar el array de resultados a un array de numpy.

2.- Graficar con este nuevo arreglo en 2d y 3d

In [ ]:
result = np.array(predictions_reduc.select('pca').collect())
reduced_data  = result[:, 0, :]
print(reduced_data.shape)

result2 = np.array(predictions_reduc.select('prediction').collect())
print(result2.shape)

center = np.array(modelKmeans.clusterCenters())
print(center)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.clf()
plt.subplot(1, 2, 1)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=result2[:,0], cmap=plt.cm.Set1, edgecolor='k')
plt.scatter(center[:, 0], center[:, 1], c='orange', s=200, alpha=0.5);
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.title("Basado en K-Means")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

plt.figure(figsize=(30,20)).gca(projection='3d').view_init(elev=90., azim=-180)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], 20, c=result2[:,0], zs=0, zdir='z')
plt.scatter(center[:, 0], center[:, 1], 300, c='orange', zs=0, zdir='z');
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.title("Basado en K-Means")

En los gráficos vemos que se logran identificar los clúster existen varios elementos que estan lejanos.

# Métricas para clústering

Entonces podemos generar algunas métricas que nos ayuden a determinar que tan buenos fueron nuestros clúster. 

Para ello vamos a ejecutar las métricas de:

1.- Con pyspark ejecutamos la métrica de accuracy, donde con los datos que se dan como `label (y_digits)` podemos comparar que tanto le hemos acertado a la estimación. 

2.- Utilizamos `sklearn` para otras métricas que son interesantes de mostrar, donde destaca la `v_mesure_score`

El etiquetado de clúster de v_mesure_score da una verdad fundamental.

La medida V es la media armónica entre homogeneidad e integridad y es independiente de los valores absolutos de las etiquetas: una permutación de los valores de las etiquetas de clase o clúster no cambiará el valor de la puntuación de ninguna manera.

Esta métrica es además simétrica: cambiar label_true con label_pred devolverá el mismo valor de puntuación. Esto puede ser útil para medir el acuerdo de dos estrategias de asignación de etiquetas independientes en el mismo conjunto de datos cuando no se conoce la verdad real.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.v_measure_score.html#sklearn.metrics.v_measure_score

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

Ambas métricas son sólo para este caso dado que clústering es un método `no supervisado` por lo que no tenemos un label contra el cual cotejar.

Primero debemos transformar los dataframe de forma:

1.- Unir dataframe de modo que queden en el mismo las variables de `prediction` y `target`. Esto se realiza con la componente de `pandas`.

2.- Crear nuevamente un dataframe en pyspark y transformar la variable `prediction` a un tipo `double`.

3.- Ejecutamos el evaluador en pyspark

4.- Ejecutamos las métricas con `sklearn`

In [ ]:
target =  pd.DataFrame()
target['target'] = y_digits
evaluacion= pd.concat([predictions_reduc.select('prediction').toPandas(), target['target']], axis=1,)

data_teval = sq.createDataFrame(evaluacion)
data_teval = data_teval.withColumn("prediction", data_teval["prediction"].cast("double"))
data_teval.printSchema()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorKM = MulticlassClassificationEvaluator(
    labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracyKM = evaluatorKM.evaluate(data_teval)
print("Test Error = %g" % (1.0 - accuracyKM))
print("Precisiòn = %g" % (accuracyKM*100))
evaluatorKM.evaluate(data_teval)

In [ ]:
from sklearn import metrics

print("Medida V de score          :", metrics.v_measure_score(evaluacion['target'], evaluacion['prediction']))
print("Medida de homogeneidad     :", metrics.homogeneity_score(evaluacion['target'], evaluacion['prediction']))
print("Medida de completitud      :", metrics.completeness_score(evaluacion['target'], evaluacion['prediction']))
print("Medida score rand ajustado :", metrics.adjusted_rand_score(evaluacion['target'], evaluacion['prediction']))
print("Medida de mutual           :", metrics.adjusted_mutual_info_score(evaluacion['target'], evaluacion['prediction']))

# Conclusión

Como se pude ver los resultados no son muy satisfactorios, entonces debemos volver a ejecutar k-means ahora con los `14` clúster y con los `10` (que sabemos que esa debiese ser la respuesta).

Otra opción es ejecutar el modelo sin la reducción de dimensionalidad o aplicando las `24` que se habían revisado.

En la realidad este (como método no supervisado) resultado se presenta al cliente final, quién nos dirá si le hacen sentido los clúster donde se han clasificado sus datos o detecta alguna inconsistencia, en cual caso debemos volver a revisar y ajustar los modelos.

# Otras formas de aplicar clústering

Podemos revisar otras formas de aplicar clústering con pyspark revisando las documentación de las componente, especial atención debemos poner en el clúster jerárquico con la componente `from pyspark.ml.clustering import BisectingKMeans`

Revisar: https://spark.apache.org/docs/latest/ml-clustering.html

`BisectingKMeans` es una especie de agrupamiento jerárquico. 
La agrupación jerárquica es uno de los métodos de análisis de agrupación más utilizados que busca construir una jerarquía de agrupaciones. Las estrategias para la agrupación jerárquica generalmente se dividen en dos tipos:

`Aglomerativo`: este es un enfoque "de abajo hacia arriba": cada observación comienza en su propio grupo, y los pares de grupos se fusionan a medida que uno avanza en la jerarquía.

`Divisivo`: este es un enfoque de "arriba hacia abajo": todas las observaciones comienzan en un grupo y las divisiones se realizan de forma recursiva a medida que uno se mueve hacia abajo en la jerarquía.

El `BisectingKMeans` es un tipo de algoritmos divisivos. 

# Generación de Dendograma

Primero para revisar el clúster aglomerativo debemos realizar un `dendograma` con los datos que nos puede decir cuantos clúster debemos considerar.
Para ello se utiliza la componente `scipy.cluster.hierarchy.linkage`, en este caso se utiliza el método `Ward`

El method ='Ward 'utiliza el algoritmo de minimización de varianza Ward entre d(u,v). Para ver fórmulas y mas datos puede consultar en:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html


In [ ]:
print(Total_Clus.show())

In [ ]:
rtest = np.array(Total_Clus.select('std_features').collect())
pruebaden  = rtest[:, 0, :]
print(pruebaden.shape)

In [ ]:
import scipy.cluster.hierarchy as sch
max_d = 80
plt.figure(figsize=(40, 15))
dendrogram = sch.dendrogram(sch.linkage(pruebaden, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.axhline(y=max_d, c='k')
plt.show() 

# Ejecución

Con el histograma podemos ver que si tomamos una distancia de `28` logramos contar con `10` clúster con los cuales ejecutaremos el algoritmo `BisectingKMeans`, posterior a ello realizaremos las transformaciones para lograr graficar los `clúster`

In [ ]:
from pyspark.ml.clustering import BisectingKMeans
kj = 8
bkm = BisectingKMeans().setK(kj).setSeed(23).setFeaturesCol('std_features').setPredictionCol('clusterJ')
modelj = bkm.fit(Total_Clus)

print("Cluster Centers: ")
centers = modelj.clusterCenters()
for center in centers:
    print(center)

predicJ = modelj.transform(Total_Clus)

Evaluar clúster jerárquico

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
cost = modelj.computeCost(Total_Clus)
print("Within Set Sum of Squared Errors = ", cost)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator().setFeaturesCol('std_features').setPredictionCol('clusterJ')

silhouette = evaluator.evaluate(predicJ)
print("Silhouette with squared euclidean distance = " + str(silhouette))


# Conclusión 2

Como se puede validar hay variados algoritmos para afrontar un problema de clústering, ahora depende mucho de la distribución de los datos que se tengan, lo que se quiere obtener, del cientista de datos y de la validación de quien ha solicitado el análisis la elección del mejor modelo.

# Referencias

Clústering definiciones y tipos: https://es.wikipedia.org/wiki/An%C3%A1lisis_de_grupos

K-means pyspark: https://spark.apache.org/docs/latest/ml-clustering.html#k-means

k-means python: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#sphx-glr-auto-examples-cluster-plot-kmeans-digits-py

Componentes principales pyspark: https://spark.apache.org/docs/latest/ml-features#pca

Componentes principales python: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

Clúster jerárquico pyspark: https://spark.apache.org/docs/latest/ml-clustering.html#bisecting-k-means

Clúster jerárquico python: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

Generación de dendograma: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

Ejemplo clústering python: https://www.aprendemachinelearning.com/k-means-en-python-paso-a-paso/

# Tarea

1.- Realizar la predicción con 10 clúster

2.- Ejecutar el Clústering para el dataset de clasificación de tipo de planta, pero sin considerar en el estudio el target (sólamente para las métricas)

`from sklearn.datasets import load_iris`

`iris = load_iris()`

In [ ]:
sc.stop()